In [27]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Thu Aug 22 15:33:16 2019

@author: smkj33
"""


# Weights for combining the similarities cased on different factors
from weights import *
from db_functions import *
import numpy as np



#-------------------------------------#
# GET RECOMMENDATION DRIVER
#-------------------------------------#

def reco_catcher_ts_ss(request_frame):
    sim = get_ts_ss_similarity(local_id) 
    content_sim = sim['scores'].apply(lambda row: pd.read_json(row, typ = 'list'))
    
    duration_weight = pd.DataFrame(request_frame['duration'])
    final_list = business_logic(duration_weight, content_sim)
    return final_list
    

def reco_catcher_cosine(request_frame):

    local_id = get_local_id(request_frame['article_id'])    
    sim = get_similarity(local_id)
    

    content_sim = sim['content_sim'].apply(lambda row: pd.read_json(row, typ = 'list'))
    title_sim = sim['title_sim'].apply(lambda row: pd.read_json(row, typ = 'list'))
    cat_tags_sim = sim['cat_tags_sim'].apply(lambda row: pd.read_json(row, typ = 'list'))

    # Similarity combination
    cosine_sim = (content_sim * float(weights_values['content_weight'])) + \
                 (title_sim * float(weights_values['title_weight']))\
                 + (cat_tags_sim * float(weights_values['cat_tags_weight']))

    cosine_sim /= (float(weights_values['content_weight']) + float(weights_values['title_weight']) + float(weights_values['cat_tags_weight']))

    duration_weight = pd.DataFrame(request_frame['duration'])
    final_list = business_logic(duration_weight, cosine_sim)
    
    return final_list


# --ADDITIONAL BUSINESS LOGIC HERE-- #
def business_logic(duration_weight, similarities):
    # In the following step, the duration values are normalised
    duration_weight = (duration_weight - duration_weight.min())/(duration_weight.max() - duration_weight.min())
    final_score = similarities.mul(duration_weight, axis = 0)

    matrix = final_score.to_numpy()
    index = np.unravel_index(np.argsort(matrix, axis=None), matrix.shape)
    index = index[1][:10]

    final_list = get_article(index)

    return final_list


In [28]:
request_frame = {'article_id': [1011, 1019, 1022],
       'duration': [120, 300, 50]}

local_id = get_local_id(request_frame['article_id'])
sim = get_ts_ss_similarity(local_id)

MySQL connection is closed
MySQL connection is closed


In [29]:
reco_catcher_ts_ss(request_frame)

MySQL connection is closed
MySQL connection is closed


,article_id,title
0,1011,Bridging the industrial heat divide
1,1019,EUSEW 2016 - energy efficiency awards
2,1020,How much can energy management actually save?
3,1021,All eyes on China’s 13th Five-Year Plan for en...
4,1628,Gothenborg Energi sees the bigger Picture
5,1629,Clean Power when ORC System recovers Waste Heat
6,1630,Significant cost savings & lowered CO2 Emissio...
7,1631,ORC-systems are helping to create a sustainabl...


In [30]:
reco_catcher_cosine(request_frame)

MySQL connection is closed
MySQL connection is closed
MySQL connection is closed


,article_id,title
0,1011,Bridging the industrial heat divide
1,1019,EUSEW 2016 - energy efficiency awards
2,1020,How much can energy management actually save?
3,1021,All eyes on China’s 13th Five-Year Plan for en...
4,1628,Gothenborg Energi sees the bigger Picture
5,1629,Clean Power when ORC System recovers Waste Heat
6,1630,Significant cost savings & lowered CO2 Emissio...
7,1631,ORC-systems are helping to create a sustainabl...


In [4]:
content_sim = sim['scores'].apply(lambda row: pd.read_json(row, typ = 'list'))

In [5]:
content_sim

,0,1,2,3,4,5,6,7,8,9,...,400,401,402,403,404,405,406,407,408,409
0,2.782518e+05,7.974746e+06,7.402850e+06,1.726674e+07,6.462720e+06,5.438599e+06,7.153903e+06,1.253049e+07,2.007954e+07,7.582098e+06,...,1.971646e+06,3.355850e+06,3.487300e+06,2.864841e+06,3.381273e+06,2.660964e+06,1.959574e+06,2.803626e+06,4.547149e+06,2.388521e+06
1,7.974746e+06,7.837502e+04,3.619516e+06,8.676816e+06,3.307931e+06,2.861647e+06,3.471539e+06,6.687141e+06,1.506041e+07,3.840055e+06,...,1.050373e+06,1.747889e+06,1.801642e+06,1.680899e+06,1.868709e+06,1.495505e+06,1.273990e+06,1.569602e+06,2.611488e+06,1.410553e+06
2,6.462720e+06,3.307931e+06,2.963097e+06,7.014830e+06,4.858882e+04,2.312559e+06,2.870595e+06,5.256163e+06,1.232788e+07,3.103148e+06,...,8.211923e+05,1.400025e+06,1.436577e+06,1.336743e+06,1.490366e+06,1.173796e+06,1.005473e+06,1.234063e+06,2.073409e+06,1.134823e+06


In [7]:
duration_weight = pd.DataFrame(request_frame['duration'])
duration_weight = (duration_weight - duration_weight.min())/(duration_weight.max() - duration_weight.min())
final_score = content_sim.mul(duration_weight, axis = 0)

In [9]:
matrix = final_score.to_numpy()
index = np.unravel_index(np.argsort(matrix, axis=None), matrix.shape)
index = index[1][:10]

final_list = get_article(index)

MySQL connection is closed


In [10]:
final_list

,article_id,title
0,1011,Bridging the industrial heat divide
1,1019,EUSEW 2016 - energy efficiency awards
2,1020,How much can energy management actually save?
3,1021,All eyes on China’s 13th Five-Year Plan for en...
4,1628,Gothenborg Energi sees the bigger Picture
5,1629,Clean Power when ORC System recovers Waste Heat
6,1630,Significant cost savings & lowered CO2 Emissio...
7,1631,ORC-systems are helping to create a sustainabl...
